### Done
* Describe Keras
* Show how to create a model in Keras

### To do
* Describe the 3 different ways of creating a model
* Save and load weights https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
* Accessing various parts of the sequential model
* Load pre-trained weights

### Notes


> TensorFlow's [contrib](https://www.tensorflow.org/versions/r0.10/api_docs/python/contrib.layers.html#layers-contrib) wraps TensorFlow functionality to support various operations such as; [convolutions](https://www.tensorflow.org/versions/r0.10/api_docs/python/contrib.layers.html#convolution2d), [batch_norm](https://www.tensorflow.org/versions/r0.10/api_docs/python/contrib.layers.html#batch_norm), [fully_connected](https://www.tensorflow.org/versions/r0.10/api_docs/python/contrib.layers.html#fully_connected).

# High-level APIs
> <span style="color:gray"> Created by Toke Faurby ([faur](https://github.com/Faur)).

Pure TF is very verbose, and it is therefore a good idea to use a high-level API of some sort.
This simplifies and speeds-up development, reduces the risk of bugs, and generally reduces headache.
Another neat benefit is that a lot of best practices (initialization, scoping, etc.) are hard-coded into the functions.

**[Keras](https://keras.io/)**
is a high-level API that can use TensorFlow or Theano as backend.
In early 2017 [TensorFlow chose Keras](http://www.fast.ai/2017/01/03/keras/) as the first high-level API to include into the tensorflow core (as opposed to contrib).
Keras has official Google support and has a large community and pre-existing examples.
Making it a good choice for high-level API to learn at the moment.

There several other high-level APIs: 
* [TFLearn](http://tflearn.org/),
* [tf.learn](https://www.tensorflow.org/get_started/tflearn) (yes, the names are confusing :( ),
* [tf.slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim), and
* [tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers).
These are generally compatible with each other, and offer overlaping functionality.
This can be confusing.
In order to simplify things we have chosen to focus on just one.

This notebook serves as a quick introduction to Keras.
As with the TF introduction this is very quick, so have a look at the **external resources** in order to get more depth:
* [Keras code](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/keras) in the TF github repo. Looking at the code is often the best way to understand what is going on. Keras also provides some good [example code](https://github.com/fchollet/keras/tree/master/examples), that demonstrates how to do many things.
* Tutorial: [Keras as a simplified interface to TensorFlow](https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)
* [TensorFlow High-Level APIs: Models in a Box](https://www.youtube.com/watch?v=t64ortpgS-E) 17 min video  (TensorFlow Dev Summit 2017)
* [Integrating Keras & TensorFlow: The Keras workflow, expanded](https://www.youtube.com/watch?v=UeheTiBJ0Io) 18 min video (TensorFlow Dev Summit 2017)


## Keras Fundamentals

Keras sits of top of `tf.layers`, and shares its implementation (e.g. `tf.layers.dense` and `keras.layers.Dense` are the same).
Keras helps with model creation, training, and evaluation.
We won't use Keras to its full potential in this course, as it sometimes encapsulates the details to such an extent that it hinders learning.

![](images/keras_overview.png)



## A Basic Example

Keras can be used in several different ways, offering varying degrees of control, encapsulation, and automation.

A lot of Keras functionality is based on the [`Model`](https:/ /keras.io/models/model/) class.
There are three ways of constructing a `Model`, which we will show now.
Each of these approaches are equivalent


In [ ]:
from __future__ import absolute_import, division, print_function 

import os
import sys
sys.path.append(os.path.join('.', '..')) 
import utils
import tensorflow as tf

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy

In [ ]:
# Load data (download if you haven't already)
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
## Create training loop
max_epoch = 250
batch_size = 64

def train(train_op, acc_value):
    init_op = tf.global_variables_initializer()
    with tf.Session().as_default() as sess:
        # Tell Keras to use the session to initialize all variables that it 
        # creates internally
        K.set_session(sess)
        sess.run(init_op)
        print('Epoch : Accuracy')
        for i in range(max_epoch):
            batch = mnist_data.train.next_batch(batch_size)
            train_op.run(feed_dict={img:batch[0], labels:batch[1]})
            if i % 10 == 0:
                accuracy = acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels})
                print ('{:5} : {:.3}'.format(i, accuracy))
    graph = utils.rename_nodes(sess.graph_def, lambda s:"/".join(s.split('_',1)))
    return graph


In [ ]:
method = 1
print('Method ' + str(method))

# Reset the graph
tf.reset_default_graph()

# Prepare placeholders
img = tf.placeholder(tf.float32, shape=(None, 784), name='img')
labels = tf.placeholder(tf.float32, shape=(None, 10), name='label')

if method == 1:
    # Easy
    # A lot of control, but 
    with tf.name_scope('model'):
        x = Dense(128, activation='relu', name='Dense1')(img)
        x = Dense(128, activation='relu', name='Dense2')(x)
    preds = Dense(10, activation='softmax', name='prediction')(x)

elif method == 2:
    # Easy
    # saving weights is easy
    # Loss of control,ugly TB graph
    with tf.name_scope('model'):
        model = Sequential()
        model.add(Dense(128, activation='relu', name='Dense1', input_dim=784)) 
        model.add(Dense(128, activation='relu', name='Dense2'))
    model.add(Dense(10, activation='softmax', name='prediction'))

    model.summary() # Print model summaries
    preds = model(img)
    
elif method == 3:
    # A bit less easy
    # Saving weights is easy
    # A lot of control    
    # Essentially method 1, wraped in a a model (similar to method 2)
    # Allows you to access intermediate layers: https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer
        with tf.name_scope('model'):
            input_layer = Input(tensor=img)
            x = Dense(128, activation='relu', name='Dense1')(input_layer)
            x = Dense(128, activation='relu', name='Dense2')(x)
        x = Dense(10, activation='softmax', name='prediction')(x)

        model = Model(inputs=input_layer, outputs=x)
        model.summary()
        preds = model.output
else:
    print('{} is not a valid method argument!'.format(method))


## Create ops
with tf.name_scope('accuracy'):
    acc_value = K.mean(categorical_accuracy(labels, preds))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(0.5)          
    train_op = optimizer.minimize(loss)


## Run training loop, and return the graph
graph = train(train_op, acc_value)

## Launch TensorBoard, and visualize the TF graph
utils.show_graph(graph)

# Credits
Created by Toke Faurby ([faur](https://github.com/Faur)).
